# Section5：Transformer

## ニューラル機械翻訳の問題点：長さに弱い
- 翻訳元の文の内容をひとつのベクトルで表現 
- 文長が長くなると表現力が足りなくなる 
- 文長と翻訳精度の関係性

## Attention (注意機構) 
- 翻訳先の各単語を選択する際に、翻訳元の文中の各単語の隠れ状態を利用

#### 翻訳元の各単語の隠れ状態の加重平均

\begin{aligned}
C_i = \sum_{j=1}^{T_x} \alpha_{ij}h_j
\end{aligned}

#### 重み(全て足すと１) 重みはFFNNで求める

\begin{aligned}
\alpha_{ij} &= \frac{exp(e_{ij})}{\sum_{k=1}^{T_x} exp(e_{ik})} \\[10px]
e_{ik} &= a(s_{i-1} , h_j)
\end{aligned}

### Attentionは何をしているのか
query(検索クエリ)に一致するkeyを索引し、対応するvalueを取り出す操作であると見做すことができる。これは辞書オブジェクトの機能と同じである。

## 注意機構には二種類ある
### Source Target-Attention(ソースターゲット注意機構)

### Self-Attention(自己注意機構)


## Transformer-Encoder

### Scaled dot product attention



### Position-Wise Feed-Forward Networks
- 位置情報を保持したまま順伝播させる

### Multi-Head attention
- 全単語に関するAttentionをまとめて計算する

#### 重みパラメタの異なる８個のヘッドを使用
- ８個のScaled Dot-Product Attentionの出力をConcat 
- それぞれのヘッドが異なる種類の情報を収集

## Decoder
- Encoderと同じく6層 
 - 各層で二種類の注意機構 
 - 注意機構の仕組みはEncoderとほぼ同じ 
- 自己注意機構 
 - 生成単語列の情報を収集 
 - 直下の層の出力へのアテンション 
 - 未来の情報を見ないようにマスク 
- Encoder-Decoder attention 
 - 入力文の情報を収集 
 - Encoderの出力へのアテンション
 
### Add (Residual Connection) 
- 入出力の差分を学習させる 
- 実装上は出力に入力をそのまま加算するだけ 
- 効果：学習・テストエラーの低減 

### Norm (Layer Normalization) 
- 各層においてバイアスを除く活性化関数への入力を平均０、分散１に正則化 
- 効果：学習の高速化



## Position Encoding
RNNを用いないので単語列の語順情報を追加する必要がある